In [1]:
import numpy as np
import cv2


In [2]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [3]:
with_mask.shape

(200, 50, 50, 3)

In [4]:
without_mask.shape

(200, 50, 50, 3)

In [5]:
with_mask = with_mask.reshape(200,50*50*3)  # turning remaining dims into columns
without_mask = without_mask.reshape(200,50*50*3 )

In [6]:
with_mask.shape

(200, 7500)

In [7]:
without_mask.shape

(200, 7500)

In [8]:
X = np.r_[with_mask, without_mask] # r_ is used in numpy to concatenate rows

In [9]:
X.shape

(400, 7500)

In [10]:
labels = np.zeros(X.shape[0])  # labeling all values to zero, X.shape[0] is 400

In [11]:
labels[200:] = 1.0  # after 200 converting to 1 as label

In [12]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [13]:
X_train,X_test,y_train,y_test = train_test_split(X, labels, test_size=0.20)


In [14]:
X_train.shape

(320, 7500)

In [15]:
# Dimensional reductionality technique to reduce 7500 columns
from sklearn.decomposition import PCA
pca = PCA(n_components=3) 
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [16]:
X_train[0] # behind maths eigen values and eigen vectors of matrix arrays in pca to reduce dims

array([ 3225.52546852, -1096.65944943,   212.28582534])

In [17]:
X_train.shape

(320, 3)

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X, labels)


In [19]:
svm = SVC(random_state=0)
svm.fit(X_train, y_train)


SVC(random_state=0)

In [20]:
y_pred = svm.predict(X_test)

In [21]:
accuracy_score(y_test,y_pred)


1.0

In [22]:
haar_data = cv2.CascadeClassifier(r"E:\AI-ML PYTHON\Project-7\Mask dataset\haarcascade_frontalface_default - Govindaraj V.xml")

In [23]:
names = {0:'Mask', 1:'No Mask'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [24]:
capture=cv2.VideoCapture(0)
data = []


while True:
    flag,img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
          #  face = pca.tranform(face)
            pred = svm.predict(face)[0]
            n = names[int(pred)] 
            cv2.rectangle(img,(x,y),(x+w, y+h), color_dict[pred] ,2)  
            cv2.rectangle(img,(x,y-40),(x+w,y), color_dict[pred],-1)
            cv2.putText(img, n, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
                                     # 1 is 16 pixes font and 2 is font weight bold
        cv2.imshow('result',img)
        
        if cv2.waitKey(2) == 27:
            break
            
capture.release()
cv2.destroyAllWindows()
